# Ransomware Analysis Automation

This notebook automates the process of executing ransomware samples in a controlled VM environment with the following workflow:
1. Restore VM to snapshot 32
2. Start the VM
3. Run event-monitoring-script.bat (auto-terminates after 1 minute)
4. Deploy and execute ransomware sample
5. Wait 50 seconds for execution
6. Kill the ransomware process
7. Copy resulting .dll files to reyhan-ransomware folder

In [1]:
import os
import time
import subprocess
import itertools
import re

In [2]:
# Create necessary directories
os.system('mkdir -p screenshots')
os.system('mkdir -p reyhan-ransomware')

0

In [3]:
# VM configuration
vm_name = 'windows11-clone'  # Update this to your VM name if different
snapshot_no = 32

# Command templates
vm_open_cmd = f'vboxmanage startvm {vm_name}'
run_cmd = f'vboxmanage guestcontrol {vm_name} run --username wineleven --password therealtreasure -- '
run_cmd += '"C:\\Windows\\System32\\cmd.exe" "/c" '
copyfrom_cmd = f'vboxmanage guestcontrol {vm_name} --username wineleven --password therealtreasure copyfrom '
kill_sample_cmd = run_cmd + f'"taskkill /f /im _sample.exe"'

# Paths and directories
output_folder = 'reyhan-ransomware'
deploy_dir = 'C:\\Users\\wineleven\\Desktop\\'
shared_folder_path = "C:\\Users\\wineleven\\Desktop\\Shared"

In [4]:
# Utility functions

def tstamp():
    """Get current timestamp in readable format."""
    t = time.time()
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(t)) + ("%.3f" % (t % 1))[1:]

def check_output_t(command):
    """Execute command and return output."""
    ret = subprocess.check_output(command, shell=True)
    return ret

def get_all_files(folder):
    """Get list of all files in a folder and its subfolders."""
    file_paths = []
    for root, _, files in os.walk(folder):
        for file in sorted(files):
            file_paths.append(os.path.join(root, file))
    return file_paths

In [5]:
# VM operations

def restore(snap_no):
    """Restore VM to specified snapshot."""
    print(f"[{tstamp()}] Restoring snapshot {snap_no}")
    check_output_t(f'vboxmanage snapshot {vm_name} restore "Snapshot {snap_no}"')
    print(f"[{tstamp()}] Restoring snapshot {snap_no} done")

def shutdown_and_restore(snap_no):
    """Power off VM and restore to snapshot."""
    print(f"[{tstamp()}] Powering off VM")
    try:
        check_output_t(f'vboxmanage controlvm {vm_name} poweroff')
        print(f"[{tstamp()}] Powering off VM done")
    except:
        print('VM (probably) already off')
    restore(snap_no)

def start_vm():
    """Start the VM."""
    print(f"[{tstamp()}] Starting up VM")
    check_output_t(vm_open_cmd)
    print(f"[{tstamp()}] Starting up VM done")
    time.sleep(5)  # Give VM time to fully start

def take_screenshot(filename):
    """Take a screenshot of the VM."""
    print(f"[{tstamp()}] Taking screenshot")
    os.system(f'vboxmanage controlvm {vm_name} screenshotpng screenshots/{filename}.png')
    print(f"[{tstamp()}] Taking screenshot done")

In [7]:
# Sample operations

def execute_monitoring_script():
    """Execute event-monitoring-script.bat in admin command prompt."""
    print(f"[{tstamp()}] Running event monitoring script")
    monitor_cmd = run_cmd + f'"event-monitoring-script.bat"'
    os.system(monitor_cmd)
    print(f"[{tstamp()}] Event monitoring script started")

def deploy_sample(filepath):
    """Copy ransomware sample to VM."""
    filename = os.path.basename(filepath)
    print(f"[{tstamp()}] Copying ransomware sample {filename}...")
    copy_cmd = run_cmd + f'"copy \"{filepath}\" \"{deploy_dir}_sample.exe\""'
    os.system(copy_cmd)
    print(f"[{tstamp()}] Copying ransomware sample done")

def execute_sample():
    """Execute the ransomware sample."""
    print(f"[{tstamp()}] Executing sample...")
    execute_cmd = run_cmd + f'"{deploy_dir}_sample.exe"'
    os.system(execute_cmd)
    print(f"[{tstamp()}] Executed sample")

def wait_sample(wait_time):
    """Wait for specified time with progress indicator."""
    print(f"[{tstamp()}] Waiting for {wait_time} seconds")
    for i in range(wait_time):
        print('\r[' + i*'#' + (wait_time-i)*'_' + ']', end='')
        time.sleep(1)
    print('\r[' + wait_time * '#' + ']')

def kill_sample():
    """Kill the ransomware sample."""
    print(f"[{tstamp()}] Killing sample...")
    os.system(kill_sample_cmd)
    print(f"[{tstamp()}] Killing sample done")

def copy_dll_to_folder():
    """Copy the .dll file from shared folder to output folder."""
    print(f"[{tstamp()}] Copying .dll files to {output_folder}")
    
    # Search for .dll files in the shared folder
    dll_search_cmd = run_cmd + f'"dir \"{shared_folder_path}\\*.dll\" /b"'
    
    try:
        dll_files = subprocess.check_output(dll_search_cmd, shell=True).decode().strip().split('\r\n')
        
        for dll_file in dll_files:
            if dll_file and len(dll_file) > 0:  # Skip empty results
                src_path = f"{shared_folder_path}\\{dll_file}"
                dst_path = f"{output_folder}/{dll_file}"
                
                # Copy file from VM to host
                copy_cmd = f'{copyfrom_cmd} "{src_path}" "{dst_path}"'
                os.system(copy_cmd)
                print(f"[{tstamp()}] Copied {dll_file} to {output_folder}")
    except subprocess.CalledProcessError as e:
        print(f"Error finding .dll files: {e}")
    
    print(f"[{tstamp()}] Copying .dll files done")

In [8]:
def run_ransomware_analysis(filepath):
    """Run the ransomware analysis workflow."""
    print(f"[{tstamp()}] Starting ransomware analysis for {filepath}")
    
    # Initialize timing
    t0 = time.time()
    
    try:
        # Restore snapshot and start VM
        restore(snapshot_no)
        start_vm()
        time.sleep(5)  # Allow system to stabilize
        
        # Execute monitoring script 
        execute_monitoring_script()
        time.sleep(2)  # Give monitoring script time to start
        
        # Deploy and execute sample (Uncomment to operate)
        # deploy_sample(filepath)
        # execute_sample()
        
        # Wait for sample to execute
        wait_sample(50)
        
        # Take screenshot for evidence (Uncomment to operate)
        # filename = os.path.basename(filepath)
        # take_screenshot(filename)
        
        # Kill sample (Uncomment to operate)
        # kill_sample()
        time.sleep(5)  # Allow system to complete operations
        
        # Copy .dll files to output folder
        # copy_dll_to_folder()
        
        # Shutdown and restore VM
        shutdown_and_restore(snapshot_no)
        
    except Exception as e:
        print(f"Error during analysis: {e}")
        shutdown_and_restore(snapshot_no)
    
    elapsed = time.time() - t0
    print(f'[{tstamp()}] Analysis completed in {elapsed:.1f} seconds.\n')

## Run Analysis for a Single Sample

Use this cell to analyze a single ransomware sample. Replace the path with your actual sample path.

In [9]:
# Path to a single ransomware sample
sample_path = "path/to/your/ransomware_sample.exe"

# Uncomment the line below to run analysis on a single sample
run_ransomware_analysis(sample_path)

[2025-05-20 10:39:48.875] Starting ransomware analysis for path/to/your/ransomware_sample.exe
[2025-05-20 10:39:48.876] Restoring snapshot 32


0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


[2025-05-20 10:39:50.446] Restoring snapshot 32 done
[2025-05-20 10:39:50.446] Starting up VM


VBoxManage: error: Out of memory condition when allocating memory with low physical backing. (VERR_NO_LOW_MEMORY)
VBoxManage: error: Details: code NS_ERROR_FAILURE (0x80004005), component ConsoleWrap, interface IConsole
VBoxManage: error: Cannot power down a saved virtual machine
VBoxManage: error: Details: code VBOX_E_INVALID_VM_STATE (0x80bb0002), component ConsoleWrap, interface IConsole, callee nsISupports
VBoxManage: error: Context: "PowerDown(progress.asOutParam())" at line 604 of file VBoxManageControlVM.cpp


Error during analysis: Command 'vboxmanage startvm windows11-clone' returned non-zero exit status 1.
[2025-05-20 10:39:53.574] Powering off VM
VM (probably) already off
[2025-05-20 10:39:53.693] Restoring snapshot 32


0%...10%...20%...30%...40%...50%...E_ACCESSDENIED
VBoxManage: error: Snapshot operation failed
VBoxManage: error: The object is not ready
VBoxManage: error: Details: code E_ACCESSDENIED (0x80070005), component SessionMachine, interface IMachine
VBoxManage: error: Context: "RTEXITCODE handleSnapshot(HandlerArg*)" at line 564 of file VBoxManageSnapshot.cpp


CalledProcessError: Command 'vboxmanage snapshot windows11-clone restore "Snapshot 32"' returned non-zero exit status 1.

## Run Analysis for Multiple Samples

Use this cell to analyze multiple ransomware samples from a folder.

In [9]:
# Path to folder containing ransomware samples
ransomware_folder = "path/to/ransomware_samples"

# Uncomment the code below to run analysis on all samples in the folder
'''
samples = get_all_files(ransomware_folder)
print(f"Found {len(samples)} samples in {ransomware_folder}")

for i, sample in enumerate(samples, 1):
    print(f"Processing sample {i}/{len(samples)}: {os.path.basename(sample)}")
    run_ransomware_analysis(sample)
'''

## Utility Functions

Additional utility functions for manual VM operations if needed.

In [10]:
def manual_restore():
    """Manually restore VM to snapshot 32."""
    restore(snapshot_no)

def manual_start():
    """Manually start the VM."""
    start_vm()
    
def manual_shutdown():
    """Manually shutdown the VM."""
    print(f"[{tstamp()}] Powering off VM")
    try:
        check_output_t(f'vboxmanage controlvm {vm_name} poweroff')
        print(f"[{tstamp()}] Powering off VM done")
    except:
        print('VM (probably) already off')
        
def check_vm_status():
    """Check if VM is running."""
    try:
        output = subprocess.check_output(f'vboxmanage showvminfo {vm_name} | grep State', shell=True).decode()
        print(output.strip())
    except:
        print("Error checking VM status")

## Manual Operations

Use these cells to perform individual operations if needed.

In [11]:
# Check VM status
# check_vm_status()

In [12]:
# Manually restore VM to snapshot 32
# manual_restore()

In [13]:
# Manually start VM
# manual_start()

In [14]:
# Manually execute monitoring script
# execute_monitoring_script()

In [15]:
# Manually copy DLL files from shared folder
# copy_dll_to_folder()

In [16]:
# Manually shutdown VM
# manual_shutdown()